In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
import os
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import func

In [2]:
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///../data/new_olympics.db'
db = SQLAlchemy(app)



C:\Users\norfe\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
sqlite3.connect('../data/olympics_raw.sqlite')
conn = sqlite3.connect('../data/new_olympics.db')

In [4]:
pd.read_sql('SELECT * FROM olympics_raw LIMIT 1', conn)

,id,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal,country_id
0,1,Athens,1896,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100m freestyle,M,Gold,1


In [6]:
 # Create engine using the `demographics.sqlite` database file
Base = automap_base()    
engine = create_engine("sqlite:///../data/new_olympics.db")
Base.prepare(engine, reflect=True)

In [7]:
Base.classes.keys()

['country_ref', 'olympics_raw']

In [8]:
Base.metadata

MetaData(bind=None)

In [9]:
class olympic_raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer)
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))

In [10]:
class raw(db.Model):
    __tablename__ = 'olympics_raw'
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    City = db.Column(db.String(64))
    Edition =  db.Column(db.String(64))
    Event= db.Column(db.String(64))
    Event_Gender= db.Column(db.String(64))
    Medal = db.Column(db.String(64))
    country_id =db.Column(db.Integer, db.ForeignKey('country_ref.id'))
    NOC = db.Column(db.String(64))
    Gender = db.Column(db.String(64))
    Discipline = db.Column(db.String(64))
    Athlete = db.Column(db.String(64))
    def __repr__(self):
        return '<raw %r>' % (self.id)

In [11]:
class country_ref(db.Model):
    __tablename__ = 'country_ref' 
    __table_args__ = {'extend_existing': True} 
    id = db.Column(db.Integer, primary_key=True)
    country_name = db.Column(db.String(64))
    code =db.Column(db.String(64))
    flag_source = db.Column(db.String(64))
    flag_image = db.Column(db.String(64))
    def __repr__(self):
        return '<country_ref %r>' % (self.id)


In [12]:
countries = Base.classes.country_ref
raw = Ba

NameError: name 'Ba' is not defined

In [13]:
cr = db.session.query(country_ref.id, country_ref.code, country_ref.country_name,country_ref.flag_image).all()


In [14]:
count = db.session.query(raw.country_id,func.count(raw.id)).group_by(raw.country_id).all()

In [15]:
subq = (db.session.query(raw.country_id,func.count(raw.id)).group_by(raw.country_id)).subquery()

In [16]:
db.session.query(subq).all()

[(1, 1053),
 (2, 146),
 (3, 145),
 (4, 4335),
 (5, 1211),
 (6, 1594),
 (7, 1314),
 (8, 1075),
 (9, 491),
 (10, 376),
 (11, 48),
 (12, 782),
 (13, 408),
 (14, 178),
 (15, 592),
 (16, 7),
 (17, 1021),
 (18, 537),
 (19, 377),
 (20, 1228),
 (21, 395),
 (22, 29),
 (23, 97),
 (24, 451),
 (25, 17),
 (26, 37),
 (27, 329),
 (28, 164),
 (29, 372),
 (30, 704),
 (31, 2),
 (32, 239),
 (33, 499),
 (34, 31),
 (35, 76),
 (36, 435),
 (37, 624),
 (38, 8),
 (39, 24),
 (40, 9),
 (41, 24),
 (42, 33),
 (43, 17),
 (44, 80),
 (45, 82),
 (46, 3),
 (47, 102),
 (48, 2),
 (49, 466),
 (50, 6),
 (51, 15),
 (52, 48),
 (53, 20),
 (54, 2049),
 (55, 11),
 (56, 331),
 (57, 4),
 (58, 260),
 (59, 17),
 (60, 121),
 (61, 23),
 (62, 5),
 (63, 42),
 (64, 38),
 (65, 21),
 (66, 16),
 (67, 1),
 (68, 4),
 (69, 7),
 (70, 81),
 (71, 84),
 (72, 825),
 (73, 490),
 (74, 6),
 (75, 22),
 (76, 19),
 (77, 52),
 (78, 11),
 (79, 1),
 (80, 21),
 (81, 1),
 (82, 2),
 (83, 1),
 (84, 23),
 (85, 679),
 (86, 1),
 (87, 2),
 (88, 4),
 (89, 14),
 (90

In [24]:
db.session.query(country_ref.country_name, country_ref.code, subq).join(subq).all()

[('Hungary', 'HUN', 1, 1053),
 ('Austria', 'AUT', 2, 146),
 ('Greece', 'GRE', 3, 145),
 ('United States', 'USA', 4, 4335),
 ('Germany', 'GER', 5, 1211),
 ('United Kingdom', 'GBR', 6, 1594),
 ('France', 'FRA', 7, 1314),
 ('Australia', 'AUS', 8, 1075),
 ('Denmark', 'DEN', 9, 491),
 ('Switzerland', 'SUI', 10, 376),
 ('Mixed Teams', 'ZZX', 11, 48),
 ('Netherlands', 'NED', 12, 782),
 ('Belgium', 'BEL', 13, 408),
 ('India', 'IND', 14, 178),
 ('Canada', 'CAN', 15, 592),
 ('Bohemia', 'BOH', 16, 7),
 ('Sweden', 'SWE', 17, 1021),
 ('Norway', 'NOR', 18, 537),
 ('Spain', 'ESP', 19, 377),
 ('Italy', 'ITA', 20, 1228),
 ('Cuba', 'CUB', 21, 395),
 ('Australasia', 'ANZ', 22, 29),
 ('South Africa', 'RSA', 23, 97),
 ('Finland', 'FIN', 24, 451),
 ('Russian Empire', 'RU1', 25, 17),
 ('Estonia', 'EST', 26, 37),
 ('Czechoslovakia', 'TCH', 27, 329),
 ('New Zealand', 'NZL', 28, 164),
 ('Brazil', 'BRA', 29, 372),
 ('Japan', 'JPN', 30, 704),
 ('Luxembourg', 'LUX', 31, 2),
 ('Argentina', 'ARG', 32, 239),
 ('Polan

In [ ]:
db.session.query(country_ref,count).filter(country_ref.id == count.country_id).all()